Inplementation of B-FRET library for our data
=============================================

[Article](https://doi.org/10.1073/pnas.2211807120), [github](https://github.com/emonetlab/B-FRET/tree/main)

In [2]:
import numpy as np
from numpy import ma
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px

from skimage import io

from domb.red_green import wt_vs_mut_multistim
from domb.utils import plot
from domb.utils.plot import CMaps
from domb.utils import masking

# Input data & parameters

In [3]:
img_DD = io.imread('/home/wisstock/bio_data/chemLTD/24_05_22/processing/09/24_05_22_09_xform_ch0.tif')
img_DA = io.imread('/home/wisstock/bio_data/chemLTD/24_05_22/processing/09/24_05_22_09_xform_ch1.tif')
img_AA = io.imread('/home/wisstock/bio_data/chemLTD/24_05_22/processing/09/24_05_22_09_xform_ch3.tif')
mask_FRET = io.imread('/home/wisstock/bio_data/chemLTD/24_05_22/processing/09/24_05_22_09_xform_Fc_norm_red-green_up-labels.tif')

a = 0.122
d = 0.794
G = 4.11

# Data chekout and profiles extracting

In [4]:
def label_prof(input_label: np.ndarray, input_img_series: np.ndarray):
    prof_I_arr = []
    prof_SD_arr=[]
    for label_num in np.unique(input_label)[1:]:
        region_mask = input_label == label_num
        prof_I = np.mean(input_img_series, axis=(1,2), where=region_mask)
        prof_SD = np.std(input_img_series, axis=(1,2), where=region_mask)

        prof_I_arr.append(prof_I)
        prof_SD_arr.append(prof_SD)
    
    return np.asarray(prof_I_arr), np.asarray(prof_SD_arr)

profiles_I_DD, profiles_SD_DD = label_prof(mask_FRET, img_DD)
profiles_I_DA, profiles_SD_DA = label_prof(mask_FRET, img_DA)
profiles_I_AA, profiles_SD_AA = label_prof(mask_FRET, img_AA)

In [19]:
I_selected_ROI = profiles_I_DD[1]
SD_selected_ROI = profiles_SD_DD[1]

time_line = np.linspace(0, I_selected_ROI.shape[0]*1, \
                        num=I_selected_ROI.shape[0])

px.line(time_line, I_selected_ROI)

# Input data formating

In [33]:
import pickle

with open('/home/wisstock/bio/scripts/ltdata/B-FRET/example/data_sinusoids.pkl', 'rb') as handle:
    b = pickle.load(handle)

In [21]:
print(b['IDD'].shape)
print(b['IDD_noise_sd'].shape)
print(b['tDD'].shape)
print(b['data_params'])
print(b['crstlk'])

(2, 1201)
(2, 1201)
(1201,)
{'DT': 8, 'AT': 18, 'CDD': 1360, 'CAA': 340, 'Emax': 0.1, 'tau_D1': 1200, 'tau_D2': 120, 'tau_A1': 3000, 'tau_A2': 300, 'delta': 0.6, 'alpha': 0.6, 'CF': 0.46}
{'d': 0.09, 'a': 0.35, 'G': 0.5}


In [30]:
selected_ROI = 1

input_data = {
    'IDD': profiles_I_DD[[0,2,3],:],
    'IDA': profiles_I_DA[[0,2,3],:],
    'IAA': profiles_I_AA[[0,2,3],:],
    'tDD': time_line,
    'tAA': time_line,
    'IDD_noise_sd': profiles_SD_DD[[0,2,3],:],
    'IDA_noise_sd': profiles_SD_DA[[0,2,3],:],
    'IAA_noise_sd': profiles_SD_AA[[0,2,3],:],
    'crstlk':{'a': a, 'd': d, 'G': G}     
}

print(input_data['IDD'].shape)
print(input_data['IDD_noise_sd'].shape)
print(input_data['tAA'].shape)
print(input_data['crstlk'])

(3, 300)
(3, 300)
(300,)
{'a': 0.122, 'd': 0.794, 'G': 4.11}


In [34]:
with open('B-FRET_test/one_roi.pkl', 'wb') as handle:
    pickle.dump(b, handle, protocol=pickle.HIGHEST_PROTOCOL)

# B-FRET

In [35]:
import sys
sys.path.append('./B-FRET/src')
from load_save import load_data
from main import analyze_FRET_data

In [36]:
DATA_DIR = r'./B-FRET_test'
EXP_DIR = r'./B-FRET_test/exp'
	
def run():

	data = load_data(DATA_DIR, 'one_roi')
	a = analyze_FRET_data(data, EXP_DIR)
	a.run()

run()

Optimizing log posterior to get mode for Laplace approx..
15158.049437549631
15158.049782478154
15158.049621044353
15158.049521413153
15158.049490784093
15158.04971023689
15158.049469100555
15158.04946113271
15158.049380750554
15158.04943945192
inf
3290737.7174034654
328079.7335459514
18093.341575872462
15017.108472698174
15017.108545120163
15017.1083034842
15017.108504209204
15017.108485220046
15017.10856519596
15017.108451319138
15017.108459977233
15017.108506043356
15017.108474582044
inf
9926220.253300427
60256.58440306081
15145.206236392283
14947.62261813815
14947.622582416925
14947.622615335758
14947.622624933163
14947.62261383578
14947.622628849123
14947.622621778699
14947.622623100173
14947.622608177995
14947.622619990245
inf
1519146.9339062062
20724.950600213386
14942.95525653657
14930.56311577393
14930.563101856445
14930.56311557294
14930.563113562786
14930.563116362091
14930.563111841053
14930.563115034565
14930.563113898748
14930.563119023349
14930.563117576814
inf
208041.41